<a href="https://colab.research.google.com/github/jzheng23/colab/blob/main/Screen_time_screen_event_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook
1. Generate new columns to make the data human-readable
2. ...

# Screen event

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Jian
google_drive_data_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Data'
google_drive_key_path = '/content/drive/MyDrive/Problematic smartphone usage/Ambient display/Key'

In [3]:
import pandas as pd
import numpy as np
import datetime
#Convert Unix timestamp to date/datetime in specified timezone
def convert_unix_to_date(unix_timestamp, timezone='America/New_York', return_date=True):
   """

   Parameters:
       unix_timestamp: Unix timestamp in milliseconds
       timezone: String of timezone (default 'America/New_York')
       return_date: If True returns date only, if False returns datetime
   """
   dt = pd.to_datetime(unix_timestamp, unit='ms', utc=True).tz_convert(timezone)
   return dt.date() if return_date else dt

In [29]:
screen_data = pd.read_csv(google_drive_data_path + '/screen_events.csv')
# screen_data = pd.read_csv(google_drive_data_path + '/Pilot/screen_events.csv')

# Calculate duration in milliseconds by simple subtraction since values are already timestamps
screen_data['duration_ms'] = screen_data['endTime'] - screen_data['startTime']
# prompt: filter out where duration_ms <= 0 in screen_data

screen_data['timezone'] = 'America/New_York'

# Calculate statDateTime using the timezone specific to each row
screen_data['startDateTime'] = screen_data.apply(
    lambda row: convert_unix_to_date(
        row['startTime'],
        timezone=row['timezone'],
        return_date=False
    ),
    axis=1
)

screen_data['endDateTime'] = screen_data.apply(
    lambda row: convert_unix_to_date(
        row['endTime'],
        timezone=row['timezone'],
        return_date=False
    ),
    axis=1
)

In [30]:
screen_data.head()

,pid,host,eventLabel,startTime,endTime,duration_ms,timezone,startDateTime,endDateTime
0,st011,589415,1,1733392561229,1733392571494,10265,America/New_York,2024-12-05 04:56:01.229000-05:00,2024-12-05 04:56:11.494000-05:00
1,st011,589415,2,1733430448708,1733433184394,2735686,America/New_York,2024-12-05 15:27:28.708000-05:00,2024-12-05 16:13:04.394000-05:00
2,st011,589415,3,1733438454648,1733438490797,36149,America/New_York,2024-12-05 17:40:54.648000-05:00,2024-12-05 17:41:30.797000-05:00
3,st011,589415,4,1733455397151,1733455436393,39242,America/New_York,2024-12-05 22:23:17.151000-05:00,2024-12-05 22:23:56.393000-05:00
4,st011,589415,5,1733553993078,1733562322519,8329441,America/New_York,2024-12-07 01:46:33.078000-05:00,2024-12-07 04:05:22.519000-05:00


In [31]:
# prompt: in screen_data, within the same pid, check if the startTime in the current row is after the endTime in the row above, save the checking result as 1 or 0 in a new column

# Create a new column 'check_result' initialized to 0
screen_data['check_result'] = 0

# Group data by 'pid'
grouped = screen_data.groupby('pid')

# Iterate through each group (pid)
for pid, group in grouped:
  # Iterate through rows within each group, starting from the second row (index 1)
  for i in range(1, len(group)):
    # Compare the startTime of the current row with the endTime of the previous row
    if group['startTime'].iloc[i] > group['endTime'].iloc[i-1]:
      # If startTime is after endTime, update 'check_result' to 1 for the current row
      screen_data.loc[group.index[i], 'check_result'] = 1

In [32]:
def ms_to_hms(ms):
    # Handle negative values
    is_negative = ms < 0
    ms = abs(ms)

    # Convert to HH:MM:SS
    seconds = int(ms / 1000)
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60

    # Format with negative sign if needed
    time_str = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    return f"-{time_str}" if is_negative else time_str

screen_data['duration'] = screen_data['duration_ms'].apply(ms_to_hms)
# prompt: In screen_data, create a new column index, with values of 1, 2, 3, 4, ...

screen_data['index'] = range(1, len(screen_data) + 1)

In [33]:
screen_data.to_csv(google_drive_data_path + '/screen_events_with_datetime.csv', index=False)

In [27]:
# prompt: for each pid, count how rows have a negative duration_ms, and how many rows have positive duration_ms

# Group data by 'pid' and calculate counts for positive and negative durations
duration_counts = screen_data.groupby('pid')['duration_ms'].agg([lambda x: (x < 0).sum(), lambda x: (x > 0).sum()])

# Rename columns for clarity
duration_counts.columns = ['negative_duration_count', 'positive_duration_count']

# Display the result
duration_counts

,negative_duration_count,positive_duration_count
pid,,
pi005,3,280
pi010,1,230
pi011,0,69
pi012,0,3
pi013,2,561
pi019,1,13
te010,0,64


In [19]:
device_data = pd.read_csv(google_drive_data_path + '/device_info.csv')

In [22]:
# prompt: merge device_data.drop(host) to duration_counts by pid

# Merge duration_counts with device_data on 'pid'
merged_data = duration_counts.merge(device_data, on='pid', how='left')

# Display the merged data
merged_data.head()

,pid,negative_duration_count,positive_duration_count,host,Base,Brand,Model
0,st011,1,19,589415.0,14.0,samsung,SM-A146U
1,st012,3,93,104241.0,14.0,samsung,SM-S911U
2,st013,40,707,926167.0,14.0,samsung,SM-S908U
3,st014,12,523,208290.0,14.0,samsung,SM-S908U1
4,st015,29,701,305317.0,13.0,Redmi,M2103K19G


In [23]:
merged_data.to_csv(google_drive_data_path + '/duration_counts.csv', index=False)

# BACKUP

##### Separate into weeks

In [ ]:
screen_events_log = screen_events_data.loc[:, ['pid', 'eventLabel', 'startTime_time', 'endTime_time']]
screen_events_log.head(5)

,pid,eventLabel,startTime_time,endTime_time
0,pi001,1,2024-11-12 16:06:40.314000-05:00,2024-11-12 16:07:58.402000-05:00
1,pi001,2,2024-11-12 16:25:36.234000-05:00,2024-11-12 16:27:51.302000-05:00
2,pi001,3,2024-11-12 17:02:36.756000-05:00,2024-11-12 17:03:39.425000-05:00
3,pi001,4,2024-11-12 17:04:12.454000-05:00,2024-11-12 17:04:24.218000-05:00
4,pi001,5,2024-11-12 17:07:08.672000-05:00,2024-11-12 17:07:20.711000-05:00


In [ ]:
# pd.set_option('display.max_rows', None)

In [ ]:
survey_meta0 = pd.read_csv(google_drive_path + '/survey_meta.csv')
survey_datetime = survey_meta0.copy()
survey_datetime['submitTime'] = survey_datetime['timestamp'].apply(lambda x: convert_unix_to_date(unix_timestamp = x, return_date= False))
survey_datetime

,pid,host,surveyID,timestamp,submitTime
0,pi001,NaN,1,1731445507251,2024-11-12 16:05:07.251000-05:00
1,pi001,NaN,2,1731689878360,2024-11-15 11:57:58.360000-05:00
2,pi002,NaN,1,1731555141456,2024-11-13 22:32:21.456000-05:00
3,pi002,NaN,2,1731715678449,2024-11-15 19:07:58.449000-05:00
4,pi003,NaN,1,1731452602347,2024-11-12 18:03:22.347000-05:00
5,pi003,NaN,2,1731722935616,2024-11-15 21:08:55.616000-05:00
6,pi003,NaN,3,1731896518856,2024-11-17 21:21:58.856000-05:00
7,pi004,NaN,1,1731586828153,2024-11-14 07:20:28.153000-05:00
8,pi004,NaN,2,1731848139596,2024-11-17 07:55:39.596000-05:00
9,pi004,NaN,3,1732102200238,2024-11-20 06:30:00.238000-05:00


In [ ]:
def ScreenEvents_by_week(data,start,end):
    pid = data["pid"]
    surveys = survey_datetime[survey_datetime["pid"] == pid]

    #survey1 = surveys[surveys["surveyID"] == 1]["submitTime"]
    survey2 = surveys[surveys["surveyID"] == 2]["submitTime"]
    survey3 = surveys[surveys["surveyID"] == 3]["submitTime"]
    #survey1_time = survey1.iloc[0] if not survey1.empty else None
    survey2_time = survey2.iloc[0] if not survey2.empty else None
    survey3_time = survey3.iloc[0] if not survey3.empty else None
    start_time = data[start]
    end_time = data[end]

    # categorizing week number
    if survey2_time and end_time < survey2_time:
        return 1
    elif survey2_time and survey3_time and start_time > survey2_time and end_time < survey3_time:
        return 2
    elif survey3_time and start_time > survey3_time:
        return 3

    # check - Surveytime = Screentime : 9
    for _, survey in surveys.iterrows():
        submit_time = survey["submitTime"]
        if start_time <= submit_time <= end_time:
            return 9
    return None

In [ ]:
screen_events_log["week"] = screen_events_log.apply(lambda row: ScreenEvents_by_week(row, "startTime_time", "endTime_time"), axis=1)
screen_events_log

,pid,eventLabel,startTime_time,endTime_time,week
0,pi001,1,2024-11-12 16:06:40.314000-05:00,2024-11-12 16:07:58.402000-05:00,1
1,pi001,2,2024-11-12 16:25:36.234000-05:00,2024-11-12 16:27:51.302000-05:00,1
2,pi001,3,2024-11-12 17:02:36.756000-05:00,2024-11-12 17:03:39.425000-05:00,1
3,pi001,4,2024-11-12 17:04:12.454000-05:00,2024-11-12 17:04:24.218000-05:00,1
4,pi001,5,2024-11-12 17:07:08.672000-05:00,2024-11-12 17:07:20.711000-05:00,1
...,...,...,...,...,...
1299,te003,59,2024-11-11 19:50:28.703000-05:00,2024-11-11 19:50:30.671000-05:00,1
1300,te003,60,2024-11-12 07:36:01.711000-05:00,2024-11-12 07:36:15.621000-05:00,3
1301,te003,61,2024-11-12 08:03:08.506000-05:00,2024-11-12 08:15:19.954000-05:00,3
1302,te003,62,2024-11-12 09:20:02.924000-05:00,2024-11-12 09:20:03.336000-05:00,3


In [ ]:
screen_events_data.to_csv('/content/screen_events_.csv', index=False)

NameError: name 'firebase_to_csv_counters' is not defined

### working on JSON data

In [ ]:
|import json
import pandas as pd
from datetime import datetime

SyntaxError: invalid syntax (<ipython-input-5-762fed006ca8>, line 1)

In [ ]:
pilot_json = open('pilot_data_1.json', encoding = 'utf-8')
data_dict = json.load(pilot_json)
data_dict

FileNotFoundError: [Errno 2] No such file or directory: 'pilot_data_1.json'

In [ ]:
all_data = []
for key, values in data_dict.items():
    for value in values:
        if value is not None:
            value_id = value.copy()
            value_id['ID'] = key
            all_data.append(value_id)
df_pilot = pd.DataFrame(all_data)
df_pilot = df_pilot[['ID', 'setting', 'timestamp', 'newValue']].rename(columns={'timestamp':'Raw_timestamp'})
df_pilot

In [ ]:
df_pilot['timestamp'] = df_pilot['Raw_timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000))  #Time format
df_pilot

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

We need a list of participants who we are going to send the reminder emails to.

In [ ]:
import pandas as pd

sign_up_data = pd.read_csv('/content/signup_survey_data.csv')

sign_up_data.head()

In [ ]:
# Insight
screen_events_data.groupby('pid')['Period_sec'].agg(['min', 'max', 'sum', 'mean', 'std'])

++ Delete the negative value rows : bc it mght be casued by reinstalling the app frequeatly while testing it.

In [ ]:
screen_events_data0 = pd.read_csv('/content/screen_events.csv')
screen_events_data = screen_events_data0.copy()
screen_events_data ["Period_sec"] = ( screen_events_data["endTime"] - screen_events_data["startTime"]) / 1000  #second
# screen_events_data ["Period_min"] = ( screen_events_data["endTime"] - screen_events_data["startTime"]) / (1000 * 60)  #minute
# screen_events_data ["Period_hour"] = ( screen_events_data["endTime"] - screen_events_data["startTime"]) / (1000 * 60 * 60) #hour
screen_events_data['startTime_time'] = pd.to_datetime(screen_events_data['startTime'], unit='ms', utc=True).dt.tz_convert('America/New_York')
screen_events_data['endTime_time'] = pd.to_datetime(screen_events_data['endTime'], unit='ms', utc=True).dt.tz_convert('America/New_York')
screen_events_data

In [ ]:

settings_change_logs_data = pd.read_csv('/content/settings_change_logs.csv')
settings_change_logs_data

In [ ]:
# Convert the timestamps
device_data = device_data0.copy()
# Convert the timezone strings to dictionaries
device_data['timezone'] = device_data['timezone'].apply(eval)

# Create new columns from the dictionary values
device_data['Base'] = device_data['timezone'].apply(lambda x: x['Base'])
device_data['Brand'] = device_data['timezone'].apply(lambda x: x['Brand'])
device_data['Model'] = device_data['timezone'].apply(lambda x: x['Model'])

# Drop the original timezone column
device_data = device_data.drop('timezone', axis=1)
device_data

,pid,host,Base,Brand,Model
0,pi005,r-8891092928c26a74-fc4w,15,google,Pixel 9
1,pi010,660225,15,google,Pixel 9
2,pi010,r-8891092928c26a74-fc4w,15,google,Pixel 9
3,pi011,r-8891092928c26a74-fc4w,15,google,Pixel 9
4,pi012,r-8891092928c26a74-fc4w,15,google,Pixel 9
5,pi013,919722,15,google,Pixel 9
6,te006,r-8891092928c26a74-v0fg,13,google,sdk_gphone64_x86_64
7,te010,r-8891092928c26a74-fc4w,15,google,Pixel 9
